# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [5]:
df = pd.read_csv("C:/Users/sushm/OneDrive/Desktop/Data_Flatiron/final_lab/dsc-course0-m8-lab/AviationData.csv", encoding='ISO-8859-1')

C:\Users\sushm\AppData\Local\Temp\ipykernel_51812\3191210931.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/sushm/OneDrive/Desktop/Data_Flatiron/final_lab/dsc-course0-m8-lab/AviationData.csv", encoding='ISO-8859-1')


In [6]:
nan_summary = df.isnull().sum()
print("NaN (missing value) count per column:\n", nan_summary)


NaN (missing value) count per column:
 Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
P

In [7]:
datatype_summary = df.dtypes
print("\nData types of each column:\n", datatype_summary)


Data types of each column:
 Event.Id                   object
Investigation.Type         object
Accident.Number            object
Event.Date                 object
Location                   object
Country                    object
Latitude                   object
Longitude                  object
Airport.Code               object
Airport.Name               object
Injury.Severity            object
Aircraft.damage            object
Aircraft.Category          object
Registration.Number        object
Make                       object
Model                      object
Amateur.Built              object
Number.of.Engines         float64
Engine.Type                object
FAR.Description            object
Schedule                   object
Purpose.of.flight          object
Air.carrier                object
Total.Fatal.Injuries      float64
Total.Serious.Injuries    float64
Total.Minor.Injuries      float64
Total.Uninjured           float64
Weather.Condition          object
Broad.phase.of.flig

In [8]:
summary_stats = df.describe()
print("\nSummary statistics for numeric columns:\n", summary_stats)


Summary statistics for numeric columns:
        Number.of.Engines  Total.Fatal.Injuries  Total.Serious.Injuries  \
count       82805.000000          77488.000000            76379.000000   
mean            1.146585              0.647855                0.279881   
std             0.446510              5.485960                1.544084   
min             0.000000              0.000000                0.000000   
25%             1.000000              0.000000                0.000000   
50%             1.000000              0.000000                0.000000   
75%             1.000000              0.000000                0.000000   
max             8.000000            349.000000              161.000000   

       Total.Minor.Injuries  Total.Uninjured  
count          76956.000000     82977.000000  
mean               0.357061         5.325440  
std                2.235625        27.913634  
min                0.000000         0.000000  
25%                0.000000         0.000000  
50%      

In [9]:
summary_stats_object = df.describe(include=['object'])
print("\nSummary statistics for object (string) columns:\n", summary_stats_object)


Summary statistics for object (string) columns:
               Event.Id Investigation.Type Accident.Number  Event.Date  \
count            88889              88889           88889       88889   
unique           87951                  2           88863       14782   
top     20001212X19172           Accident      CEN22LA149  1984-06-30   
freq                 3              85015               2          25   

             Location        Country Latitude Longitude Airport.Code  \
count           88837          88663    34382     34373        50132   
unique          27758            219    25592     27156        10374   
top     ANCHORAGE, AK  United States  332739N  0112457W         NONE   
freq              434          82248       19        24         1488   

       Airport.Name  ... Amateur.Built    Engine.Type FAR.Description  \
count         52704  ...         88787          81793           32023   
unique        24870  ...             2             12              31   
top 

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [10]:
print("\nUnique Makes:", df['Make'].nunique())
print("Year/Date columns:", [col for col in df.columns if 'year' in col.lower() or 'date' in col.lower()])


Unique Makes: 8237
Year/Date columns: ['Event.Date', 'Publication.Date']


In [11]:
if 'Year' in df.columns:
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
elif 'Event.Date' in df.columns:
    df['Year'] = pd.to_datetime(df['Event.Date'], errors='coerce').dt.year
df = df[df['Year'] >= 1983]

In [12]:
injury_cols = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Total.Occupants']
for col in injury_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [13]:
if 'Total.Occupants' not in df.columns and all(col in df.columns for col in ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']):
    df['Total.Occupants'] = df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + df['Total.Minor.Injuries'] + df['Total.Uninjured']

In [48]:
df['Frac.Fatal.Serious'] = 0.0  # Make column float from the start
mask = df['Total.Occupants'] > 0
df.loc[mask, 'Frac.Fatal.Serious'] = (
    df.loc[mask, 'Total.Fatal.Injuries'] + df.loc[mask, 'Total.Serious.Injuries']
) / df.loc[mask, 'Total.Occupants']

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [49]:
print(df.columns)


Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date', 'Year',
       'Total.Occupants', 'Frac.Fatal.Serious', 'damage', 'Engine_type'],
      dtype='object')


In [50]:
df.columns = df.columns.str.strip()
df['damage'] = df['Aircraft.damage'].str.strip().str.title()

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [51]:
print("\nTop 20 Makes:\n", df['Make'].value_counts().head(20))


Top 20 Makes:
 Make
CESSNA               25814
PIPER                14142
BEECH                 5109
BOEING                2696
BELL                  2610
MOONEY                1278
ROBINSON              1207
GRUMMAN               1068
BELLANCA               983
HUGHES                 879
SCHWEIZER              756
AIR TRACTOR            673
AERONCA                607
MCDONNELL DOUGLAS      603
MAULE                  572
CHAMPION               507
STINSON                421
AERO COMMANDER         412
DE HAVILLAND           407
LUSCOMBE               391
Name: count, dtype: int64


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [52]:
df['Make'] = df['Make'].str.strip().str.upper().fillna('UNKNOWN')



In [53]:
# Filtering: Keep makes with >= 50 accidents in this period
make_counts = df['Make'].value_counts()
valid_makes = make_counts[make_counts >= 50].index
df = df[df['Make'].isin(valid_makes)]


In [54]:
# --- Inspect Model column ---
print("\nNumber of unique Models:", df['Model'].nunique())
print(df['Model'].value_counts().head(20))


Number of unique Models: 6310
Model
152          2231
172          1654
172N         1096
PA-28-140     869
172M          760
150           752
172P          665
182           618
180           596
150M          553
PA-18         550
PA-18-150     550
PA-28-180     548
PA-28-161     526
PA-28-181     509
737           489
206B          488
150L          436
A36           428
PA-38-112     426
Name: count, dtype: int64


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [55]:
# Engine.Type
df.loc[:, 'Engine_type'] = df['Engine.Type'].str.strip().str.upper().fillna('UNKNOWN')

In [56]:
# Weather.Condition
df.loc[:, 'Weather.Condition'] = df['Weather.Condition'].str.strip().str.upper().fillna('UNKNOWN')


In [57]:
# Number.of.Engines
df.loc[:, 'Number.of.Engines'] = pd.to_numeric(df['Number.of.Engines'], errors='coerce').astype('Int64')

In [58]:
# Purpose.of.flight
df.loc[:, 'Purpose.of.flight'] = df['Purpose.of.flight'].str.strip().str.upper().fillna('UNKNOWN')

In [59]:
# Broad.phase.of.flight
df.loc[:, 'Broad.phase.of.flight'] = df['Broad.phase.of.flight'].str.strip().str.upper().fillna('UNKNOWN')

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [60]:
# Drop columns with >70% NaNs
threshold = 0.7
na_frac = df.isna().mean()
drop_cols = na_frac[na_frac > threshold].index
print(f"\nDropping {len(drop_cols)} columns with >70% NaNs:", list(drop_cols))
df = df.drop(columns=drop_cols)



Dropping 0 columns with >70% NaNs: []


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [61]:
df.to_csv('C:/Users/sushm/OneDrive/Desktop/Data_Flatiron/final_lab/dsc-course0-m8-lab/data/aviation_accidents_cleaned.csv', index=False)

In [62]:
print("Saved cleaned dataframe to 'C:/Users/sushm/OneDrive/Desktop/Data_Flatiron/final_lab/dsc-course0-m8-lab/data/aviation_accidents_cleaned.csv'")

Saved cleaned dataframe to 'C:/Users/sushm/OneDrive/Desktop/Data_Flatiron/final_lab/dsc-course0-m8-lab/data/aviation_accidents_cleaned.csv'
